Notebook: AI Text Generator.

**1. Introduction: Text Generation using Neural Networks**

In this project, we explore the realm of neural networks and their application in generating text. The primary goal was to create a model capable of generating coherent and contextually relevant phrases, building upon the patterns found in a given dataset (here, a poem about cookies).

**2. Libraries and Technologies Used:**

The project leverages the power of Python and several key libraries to implement and train the neural network. Notably, it employs NumPy for numerical operations, and PyTorch for building and training the neural network.

**3. Project Overview:**

The initial step involved preprocessing a text dataset by converting it to lowercase, extracting unique characters, and assigning numerical values to each character. This laid the foundation for transforming textual information into a format suitable for neural network training.

The model architecture consists of an ensemble of four attention-based heads, each contributing to the learning process. These heads focus on capturing different aspects of the input sequence. The model then progresses through layers of fully connected neural networks, ultimately generating a prediction for the next character in the sequence.

**4. Training and Optimization:**

The training process involves iteratively feeding sequences of characters into the model, optimizing the model parameters with respect to the desired predictions. The loss function used for optimization is cross-entropy, a suitable choice for sequence generation tasks.

An Adam optimizer is employed to efficiently update the weights of the model during training. The training loop runs for a predefined number of iterations, with periodic updates on the loss for monitoring the model's progress.

**5. Results:**

As the model evolves through training iterations, it gains the ability to generate new text sequences. The project concludes with a demonstration of the model's capabilities by generating a text sequence, showcasing the potential for creative and contextually relevant outputs.

For a detailed, step-by-step explanation of the code, see neural_network.ipynb file.


In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

# To open a text file, run the following:
# with open('file.txt'. 'r'. encoding='utf-8') as f:
#     text = f.read()

text = '''

In the oven's warm embrace they lie,
A symphony of scents begins to fly.
Buttery whispers, sugary notes,
A dance of flavors, the cookie gloats.

Golden brown, a caramel delight,
A treat to savor, a delicious bite.
Chocolate chips, like stars in the night,
Melting away, a sweet delight.

Cinnamon swirls in a fragrant breeze,
A cookie masterpiece, sure to please.
Soft and chewy, or crisp and thin,
Each creation a sweet little win.

Oatmeal clusters, a hearty blend,
A journey of taste that has no end.
Almond accents, nutty and bold,
In every nibble, a story told.

Sugar dusted, a powdered grace,
A sprinkle of joy on a sugary base.
Vanilla dreams, a comforting hug,
Wrapped in dough, like a cozy rug.

So, take a bite of this cookie song,
A delightful chorus that won't be long.
In the world of treats, they reign supreme,
Oh, sweet cookies, a heavenly dream.

But wait, there's more in the cookie jar,
A symphony of flavors, near and far.
Macadamia melodies, a crunchy delight,
Mingling with coconut, oh, what a sight.

Peanut butter echoes, smooth and rich,
A delectable chorus, an irresistible pitch.
Raisin rhythms, a sweet surprise,
In the cookie dance, they harmonize.

Lemon zest, a citrusy kiss,
Adds a zing to the cookie bliss.
Ginger sparks, a warming hue,
A flavor adventure, both old and new.

Minty freshness, a cool refrain,
Peppermint pleasures, a candy cane.
Maple whispers, autumn's embrace,
In the cookie kingdom, a royal grace.

Butterscotch tales, a caramel swirl,
A caramel symphony, a tempting twirl.
Hazelnut dreams, a nutty trance,
In the world of cookies, a sweet advance.

So, as you explore this cookie sea,
Let your taste buds wander, wild and free.
In every crumb and every seam,
A cookie world, a delicious dream.

'''

# Preprocessing, change to lower case to reduce vocab size
text = text.lower()
# Get letters
chars = sorted(list(set(text)))
# Assign a number to each letter in a dictionary
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

# Convert text to numbers
data = [stoi[c] for c in text]
vocab_size = len(chars)

ins = 16
outs = vocab_size
nodes = 100
lr = 0.001

n_emb = 32
embed = torch.randn(vocab_size, n_emb)
pos = torch.randn(ins, n_emb)

data = torch.tensor(data).long()


def generate_text(s, length):
    print("generated text:")
    print("---------------------")
    gen_text = ""
    for i in range(length):
        yh = model.forward(s)
        prob = F.softmax(yh[-1, :], dim=0)
        # pred = torch.argmax(yh).item()
        pred = torch.multinomial(prob, num_samples=1).item()        
        s = torch.roll(s, -1)
        s[-1] = pred
        gen_text += itos[pred]
    
    print(gen_text)

params = [] 
def weights(ins, outs):
    ws = torch.randn(ins, outs)*0.1
    ws.requires_grad_(True)
    params.append(ws)
    return ws

class Head():
    def __init__(self):
        self.wv = weights(n_emb, n_emb//4)
        self.wr = weights(n_emb, ins)
    
    def forward(self, x):
        v = x @ self.wv
        attn = x @ self.wr
        tril = torch.tril(attn)
        tril = tril.masked_fill(tril == 0, -1e10)
        rew = F.softmax(tril, dim=-1)
        x = rew @ v
        return x        

class Model():
    def __init__(self):
        self.heads = [Head(), Head(), Head(), Head()]
        self.w0 = weights(n_emb, nodes)
        self.w1 = weights(nodes, nodes)
        self.w2 = weights(nodes, outs)
        
    def forward(self, x):
        x = embed[x] + pos
        x = torch.cat([head.forward(x) for head in self.heads], dim=-1)        
        x = torch.relu(x @ self.w0)
        x = torch.relu(x @ self.w1)
        yh = x @ self.w2    
        return yh
    
model = Model()
optimizer = torch.optim.Adam(params, lr)

for it in range(5000):
    
    b = torch.randint(len(data)-ins, (100,))
    xs = torch.stack([data[i:i+ins] for i in b])
    ys = torch.stack([data[i+1:i+ins+1] for i in b])

    yh = model.forward(xs)
    loss = F.cross_entropy(yh.view(-1, vocab_size) , ys.long().view(-1)) 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    e = loss.item()
    if it%1000==0:
        print(it, "loss:", e)
            
generate_text(xs[0], 1000)

0 loss: 3.3980443477630615
1000 loss: 1.3685879707336426
2000 loss: 0.7451860308647156
3000 loss: 0.5296571254730225
4000 loss: 0.45431315898895264
generated text:
---------------------
 bite of this coon supreme, base.
vanilla dreams, a warming hue,
a symphony, a sweet little win.

oa
